In [1]:
#!pip install sqlalchemy

In [2]:
import sqlalchemy as sqla
from sqlalchemy import create_engine
import traceback 
import glob
import os 
import pprint
from pprint import pprint
import json
import requests
import time
from IPython.display import display

In [3]:
url = "dbbikes.cvvkn6jkfbdf.eu-west-1.rds.amazonaws.com"
port = "3306"
db = "dbbikes"
user = "SEGroup25"
password = "SEGroup25?"

In [4]:
engine = create_engine(f"mysql+mysqldb://{user}:{password}@{url}:{port}/{db}", echo=True)

In [5]:
for res in engine.execute("SHOW VARIABLES"):
    print(res)

2023-03-03 15:07:51,701 INFO sqlalchemy.engine.Engine SELECT DATABASE()
2023-03-03 15:07:51,702 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-03-03 15:07:51,713 INFO sqlalchemy.engine.Engine SELECT @@sql_mode
2023-03-03 15:07:51,715 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-03-03 15:07:51,722 INFO sqlalchemy.engine.Engine SELECT @@lower_case_table_names
2023-03-03 15:07:51,723 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-03-03 15:07:51,735 INFO sqlalchemy.engine.Engine SHOW VARIABLES
2023-03-03 15:07:51,737 INFO sqlalchemy.engine.Engine [raw sql] ()
('activate_all_roles_on_login', 'OFF')
('admin_address', '')
('admin_port', '33062')
('admin_ssl_ca', '')
('admin_ssl_capath', '')
('admin_ssl_cert', '')
('admin_ssl_cipher', '')
('admin_ssl_crl', '')
('admin_ssl_crlpath', '')
('admin_ssl_key', '')
('admin_tls_ciphersuites', '')
('admin_tls_version', 'TLSv1.2,TLSv1.3')
('authentication_policy', '*')
('auto_generate_certs', 'ON')
('auto_increment_increment', '1')
('auto_increment_

In [6]:


    
#Drop table
engine.execute("DROP TABLE IF EXISTS `dbbikes`.`stations`")

#Create table

sql = """
CREATE TABLE `dbbikes`.`stations` (
  `number` INT NOT NULL,
  `contract_name` VARCHAR(45) NULL,
  `stat_name` VARCHAR(45) NULL,
  `address` VARCHAR(45) NULL,
  `position_lat` REAL NULL,
  `position_long` REAL NULL,
  `banking` TINYINT NULL,
  `bonus` TINYINT NULL,
  `bikestands` VARCHAR(45) NULL,
  `stat_status` VARCHAR(45) NULL,
  PRIMARY KEY (`number`));
"""
engine.execute(sql)

#Truncate table
  
#sql = """
 #   TRUNCATE TABLE `dbbikes`.`stations`

#"""
#engine.execute(sql)


2023-03-03 15:07:51,773 INFO sqlalchemy.engine.Engine DROP TABLE IF EXISTS `dbbikes`.`stations`
2023-03-03 15:07:51,774 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-03-03 15:07:51,794 INFO sqlalchemy.engine.Engine COMMIT
2023-03-03 15:07:51,809 INFO sqlalchemy.engine.Engine 
CREATE TABLE `dbbikes`.`stations` (
  `number` INT NOT NULL,
  `contract_name` VARCHAR(45) NULL,
  `stat_name` VARCHAR(45) NULL,
  `address` VARCHAR(45) NULL,
  `position_lat` REAL NULL,
  `position_long` REAL NULL,
  `banking` TINYINT NULL,
  `bonus` TINYINT NULL,
  `bikestands` VARCHAR(45) NULL,
  `stat_status` VARCHAR(45) NULL,
  PRIMARY KEY (`number`));

2023-03-03 15:07:51,810 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-03-03 15:07:51,844 INFO sqlalchemy.engine.Engine COMMIT


In [7]:
sql = """
CREATE TABLE IF NOT EXISTS availability (
available_bike_stands INT,
available_bikes INT,
last_update INT,
number INT)
"""

try:
    res = engine.execute(sql)
    print(res.fetchall())
except Exception as e:
    print(e)

2023-03-03 15:07:51,887 INFO sqlalchemy.engine.Engine 
CREATE TABLE IF NOT EXISTS availability (
available_bike_stands INT,
available_bikes INT,
last_update INT,
number INT)

2023-03-03 15:07:51,888 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-03-03 15:07:51,897 INFO sqlalchemy.engine.Engine COMMIT
This result object does not return rows. It has been closed automatically.


In [8]:
# meta_data = sqla.MetaData(bind=engine)
# sqla.MetaData.reflect(meta_data)
# sTable = meta_data.tables['station']

In [9]:
import requests
import traceback
import datetime
import time 

In [10]:
APIKEY = "5ac1b590c7e338d66c34fa0c5610e94c99a3b45c"
NAME = "Dublin"
STATIONS = "https://api.jcdecaux.com/vls/v1/stations"

In [ ]:
#r = requests.get(STATIONS , params = {"apiKey": APIKEY, "contract": NAME})
def write_to_file(text):
    now = datetime.datetime.now()
    with open(f"data/bikes_{now}".replace(" ", "_"), "w") as f:
        f.write(text)
    

            
def stations_to_db(text):
    stations = json.loads(text)
    print(type(stations),len(stations))
    print(text)

    for station in stations:
        print(station)
        vals = (
            station.get("number"),
            station.get("contract_name"),
            station.get("name"),
            station.get("address"),
            station.get("position").get("lat"),
            station.get("position").get("lng"),
            station.get("banking"),
            station.get("bonus"),
            station.get("bike_stands"),
            station.get("status")
        )
        print("\n\n\nVALS:{}\n\n\n".format(vals))
        engine.execute("insert into stations values(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)", vals)
        #break

def availability_to_db(text):
    availability =json.loads(text)
    print(type(availability), len(availability))
    for available in availability:
        print(available)
        vals = (
        (available.get("available_bike_stands")),
        (available.get("available_bikes")),
        available.get("last_update"),
        (available.get("number"))
        )
        engine.execute("insert into availability values(%s, %s, %s, %s)", vals)
        #break
    return


#availability_to_db(r.text)

def main():
    while True:
        try:
            now = datetime.datetime.now()
            r = requests.get(STATIONS , params = {"apiKey": APIKEY, "contract": NAME})
            print(r,now)
            #write_to_file(r.text)
            stations_to_db(r.text)
            availability_to_db(r.text)
            time.sleep(5 * 60)
        except:
            print(traceback.format_exc())
            if engine is None:
                return

if __name__ == "__main__":
    main()  

<Response [200]> 2023-03-03 15:07:51.945909
<class 'list'> 115
[{"number":42,"contract_name":"dublin","name":"SMITHFIELD NORTH","address":"Smithfield North","position":{"lat":53.349562,"lng":-6.278198},"banking":false,"bonus":false,"bike_stands":30,"available_bike_stands":21,"available_bikes":9,"status":"OPEN","last_update":1677856059000},{"number":30,"contract_name":"dublin","name":"PARNELL SQUARE NORTH","address":"Parnell Square North","position":{"lat":53.3537415547453,"lng":-6.26530144781526},"banking":false,"bonus":false,"bike_stands":20,"available_bike_stands":13,"available_bikes":7,"status":"OPEN","last_update":1677855811000},{"number":54,"contract_name":"dublin","name":"CLONMEL STREET","address":"Clonmel Street","position":{"lat":53.336021,"lng":-6.26298},"banking":false,"bonus":false,"bike_stands":33,"available_bike_stands":20,"available_bikes":13,"status":"OPEN","last_update":1677855762000},{"number":108,"contract_name":"dublin","name":"AVONDALE ROAD","address":"Avondale Road

2023-03-03 15:07:52,120 INFO sqlalchemy.engine.Engine [raw sql] (42, 'dublin', 'SMITHFIELD NORTH', 'Smithfield North', 53.349562, -6.278198, False, False, 30, 'OPEN')
2023-03-03 15:07:52,151 INFO sqlalchemy.engine.Engine COMMIT
{'number': 30, 'contract_name': 'dublin', 'name': 'PARNELL SQUARE NORTH', 'address': 'Parnell Square North', 'position': {'lat': 53.3537415547453, 'lng': -6.26530144781526}, 'banking': False, 'bonus': False, 'bike_stands': 20, 'available_bike_stands': 13, 'available_bikes': 7, 'status': 'OPEN', 'last_update': 1677855811000}



VALS:(30, 'dublin', 'PARNELL SQUARE NORTH', 'Parnell Square North', 53.3537415547453, -6.26530144781526, False, False, 20, 'OPEN')



2023-03-03 15:07:52,172 INFO sqlalchemy.engine.Engine insert into stations values(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
2023-03-03 15:07:52,175 INFO sqlalchemy.engine.Engine [raw sql] (30, 'dublin', 'PARNELL SQUARE NORTH', 'Parnell Square North', 53.3537415547453, -6.26530144781526, False, False, 20, 'OPEN

2023-03-03 15:07:52,583 INFO sqlalchemy.engine.Engine [raw sql] (13, 'dublin', 'FITZWILLIAM SQUARE WEST', 'Fitzwilliam Square West', 53.336074, -6.252825, False, False, 30, 'OPEN')
2023-03-03 15:07:52,592 INFO sqlalchemy.engine.Engine COMMIT
{'number': 43, 'contract_name': 'dublin', 'name': 'PORTOBELLO ROAD', 'address': 'Portobello Road', 'position': {'lat': 53.330091, 'lng': -6.268044}, 'banking': False, 'bonus': False, 'bike_stands': 30, 'available_bike_stands': 27, 'available_bikes': 3, 'status': 'OPEN', 'last_update': 1677855648000}



VALS:(43, 'dublin', 'PORTOBELLO ROAD', 'Portobello Road', 53.330091, -6.268044, False, False, 30, 'OPEN')



2023-03-03 15:07:52,612 INFO sqlalchemy.engine.Engine insert into stations values(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
2023-03-03 15:07:52,614 INFO sqlalchemy.engine.Engine [raw sql] (43, 'dublin', 'PORTOBELLO ROAD', 'Portobello Road', 53.330091, -6.268044, False, False, 30, 'OPEN')
2023-03-03 15:07:52,624 INFO sqlalchemy.engine.Engine COMM

2023-03-03 15:07:52,924 INFO sqlalchemy.engine.Engine [raw sql] (87, 'dublin', 'COLLINS BARRACKS MUSEUM', 'Collins Barracks Museum', 53.347477, -6.28525, False, False, 38, 'OPEN')
2023-03-03 15:07:52,929 INFO sqlalchemy.engine.Engine COMMIT
{'number': 84, 'contract_name': 'dublin', 'name': 'BROOKFIELD ROAD', 'address': 'Brookfield Road', 'position': {'lat': 53.339005, 'lng': -6.300217}, 'banking': False, 'bonus': False, 'bike_stands': 30, 'available_bike_stands': 10, 'available_bikes': 20, 'status': 'OPEN', 'last_update': 1677855473000}



VALS:(84, 'dublin', 'BROOKFIELD ROAD', 'Brookfield Road', 53.339005, -6.300217, False, False, 30, 'OPEN')



2023-03-03 15:07:52,948 INFO sqlalchemy.engine.Engine insert into stations values(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
2023-03-03 15:07:52,949 INFO sqlalchemy.engine.Engine [raw sql] (84, 'dublin', 'BROOKFIELD ROAD', 'Brookfield Road', 53.339005, -6.300217, False, False, 30, 'OPEN')
2023-03-03 15:07:52,992 INFO sqlalchemy.engine.Engine COMM

2023-03-03 15:07:53,265 INFO sqlalchemy.engine.Engine [raw sql] (99, 'dublin', 'CITY QUAY', 'City Quay', 53.346637, -6.246154, False, False, 30, 'OPEN')
2023-03-03 15:07:53,272 INFO sqlalchemy.engine.Engine COMMIT
{'number': 9, 'contract_name': 'dublin', 'name': 'EXCHEQUER STREET', 'address': 'Exchequer Street', 'position': {'lat': 53.343034, 'lng': -6.263578}, 'banking': False, 'bonus': False, 'bike_stands': 24, 'available_bike_stands': 13, 'available_bikes': 11, 'status': 'OPEN', 'last_update': 1677855879000}



VALS:(9, 'dublin', 'EXCHEQUER STREET', 'Exchequer Street', 53.343034, -6.263578, False, False, 24, 'OPEN')



2023-03-03 15:07:53,290 INFO sqlalchemy.engine.Engine insert into stations values(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
2023-03-03 15:07:53,290 INFO sqlalchemy.engine.Engine [raw sql] (9, 'dublin', 'EXCHEQUER STREET', 'Exchequer Street', 53.343034, -6.263578, False, False, 24, 'OPEN')
2023-03-03 15:07:53,297 INFO sqlalchemy.engine.Engine COMMIT
{'number': 67, 'contr

...

2023-03-03 15:07:56,530 INFO sqlalchemy.engine.Engine [raw sql] (12, 28, 1677855943000, 58)
2023-03-03 15:07:56,536 INFO sqlalchemy.engine.Engine COMMIT
{'number': 66, 'contract_name': 'dublin', 'name': 'NEW CENTRAL BANK', 'address': 'New Central Bank', 'position': {'lat': 53.347122, 'lng': -6.234749}, 'banking': False, 'bonus': False, 'bike_stands': 40, 'available_bike_stands': 19, 'available_bikes': 20, 'status': 'OPEN', 'last_update': 1677855791000}
2023-03-03 15:07:56,552 INFO sqlalchemy.engine.Engine insert into availability values(%s, %s, %s, %s)
2023-03-03 15:07:56,552 INFO sqlalchemy.engine.Engine [raw sql] (19, 20, 1677855791000, 66)
2023-03-03 15:07:56,557 INFO sqlalchemy.engine.Engine COMMIT
{'number': 104, 'contract_name': 'dublin', 'name': 'GRANGEGORMAN LOWER (CENTRAL)', 'address': 'Grangegorman Lower (Central)', 'position': {'lat': 53.355173, 'lng': -6.278424}, 'banking': False, 'bonus': False, 'bike_stands': 40, 'available_bike_stands': 28, 'available_bikes': 12, 'status

2023-03-03 15:07:56,878 INFO sqlalchemy.engine.Engine [raw sql] (19, 21, 1677856038000, 69)
2023-03-03 15:07:56,884 INFO sqlalchemy.engine.Engine COMMIT
{'number': 25, 'contract_name': 'dublin', 'name': 'MERRION SQUARE EAST', 'address': 'Merrion Square East', 'position': {'lat': 53.339434, 'lng': -6.246548}, 'banking': False, 'bonus': False, 'bike_stands': 30, 'available_bike_stands': 18, 'available_bikes': 12, 'status': 'OPEN', 'last_update': 1677855841000}
2023-03-03 15:07:56,898 INFO sqlalchemy.engine.Engine insert into availability values(%s, %s, %s, %s)
2023-03-03 15:07:56,899 INFO sqlalchemy.engine.Engine [raw sql] (18, 12, 1677855841000, 25)
2023-03-03 15:07:56,904 INFO sqlalchemy.engine.Engine COMMIT
{'number': 51, 'contract_name': 'dublin', 'name': 'YORK STREET WEST', 'address': 'York Street West', 'position': {'lat': 53.339334, 'lng': -6.264699}, 'banking': False, 'bonus': False, 'bike_stands': 40, 'available_bike_stands': 36, 'available_bikes': 4, 'status': 'OPEN', 'last_upd

2023-03-03 15:07:57,234 INFO sqlalchemy.engine.Engine [raw sql] (39, 1, 1677855923000, 78)
2023-03-03 15:07:57,240 INFO sqlalchemy.engine.Engine COMMIT
{'number': 2, 'contract_name': 'dublin', 'name': 'BLESSINGTON STREET', 'address': 'Blessington Street', 'position': {'lat': 53.356769, 'lng': -6.26814}, 'banking': False, 'bonus': False, 'bike_stands': 20, 'available_bike_stands': 16, 'available_bikes': 4, 'status': 'OPEN', 'last_update': 1677855990000}
2023-03-03 15:07:57,258 INFO sqlalchemy.engine.Engine insert into availability values(%s, %s, %s, %s)
2023-03-03 15:07:57,258 INFO sqlalchemy.engine.Engine [raw sql] (16, 4, 1677855990000, 2)
2023-03-03 15:07:57,265 INFO sqlalchemy.engine.Engine COMMIT
{'number': 75, 'contract_name': 'dublin', 'name': 'JAMES STREET', 'address': 'James Street', 'position': {'lat': 53.343456, 'lng': -6.287409}, 'banking': False, 'bonus': False, 'bike_stands': 40, 'available_bike_stands': 38, 'available_bikes': 2, 'status': 'OPEN', 'last_update': 1677856028

2023-03-03 15:07:57,592 INFO sqlalchemy.engine.Engine [raw sql] (26, 9, 1677855939000, 85)
2023-03-03 15:07:57,599 INFO sqlalchemy.engine.Engine COMMIT
{'number': 107, 'contract_name': 'dublin', 'name': 'CHARLEVILLE ROAD', 'address': 'Charleville Road', 'position': {'lat': 53.359157, 'lng': -6.281866}, 'banking': False, 'bonus': False, 'bike_stands': 40, 'available_bike_stands': 30, 'available_bikes': 10, 'status': 'OPEN', 'last_update': 1677855783000}
2023-03-03 15:07:57,612 INFO sqlalchemy.engine.Engine insert into availability values(%s, %s, %s, %s)
2023-03-03 15:07:57,613 INFO sqlalchemy.engine.Engine [raw sql] (30, 10, 1677855783000, 107)
2023-03-03 15:07:57,620 INFO sqlalchemy.engine.Engine COMMIT
{'number': 33, 'contract_name': 'dublin', 'name': "PRINCES STREET / O'CONNELL STREET", 'address': "Princes Street / O'Connell Street", 'position': {'lat': 53.349013, 'lng': -6.260311}, 'banking': True, 'bonus': False, 'bike_stands': 23, 'available_bike_stands': 21, 'available_bikes': 2,

2023-03-03 15:07:57,927 INFO sqlalchemy.engine.Engine [raw sql] (27, 13, 1677856036000, 92)
2023-03-03 15:07:57,934 INFO sqlalchemy.engine.Engine COMMIT
{'number': 21, 'contract_name': 'dublin', 'name': 'LEINSTER STREET SOUTH', 'address': 'Leinster Street South', 'position': {'lat': 53.34218, 'lng': -6.254485}, 'banking': False, 'bonus': False, 'bike_stands': 30, 'available_bike_stands': 18, 'available_bikes': 12, 'status': 'OPEN', 'last_update': 1677855525000}
2023-03-03 15:07:57,953 INFO sqlalchemy.engine.Engine insert into availability values(%s, %s, %s, %s)
2023-03-03 15:07:57,956 INFO sqlalchemy.engine.Engine [raw sql] (18, 12, 1677855525000, 21)
2023-03-03 15:07:57,963 INFO sqlalchemy.engine.Engine COMMIT
{'number': 88, 'contract_name': 'dublin', 'name': 'BLACKHALL PLACE', 'address': 'Blackhall Place', 'position': {'lat': 53.3488, 'lng': -6.281637}, 'banking': False, 'bonus': False, 'bike_stands': 30, 'available_bike_stands': 20, 'available_bikes': 10, 'status': 'OPEN', 'last_upd

In [ ]:
#r = requests.get(STATIONS , params = {"apiKey": APIKEY, "contract": NAME})

In [ ]:
#text = r.text

In [ ]:
#stations = json.loads(text)

In [ ]:

"""
for station in stations[:10]:

        vals = (
            station.get("number"),
            station.get("contract_name"),
            station.get("name"),
            station.get("address"),
            station.get("position").get("lat"),
            station.get("position").get("lng"),
            station.get("banking"),
            station.get("bonus"),
            station.get("bike_stands"),
            station.get("status")
        )
        engine.execute("insert into stations values(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)", vals)
        break
"""


In [ ]:
#number, name, address, latitude, longitude, bike_stands